In [1]:
# Set up the package development environment (e.g. after restarting kernel)
# pwd() should be "C:\\GitWorking\\Autologistic"
using Pkg  
Pkg.activate(pwd())   #At cmd line, could use ]activate .
using Revise
using Autologistic

┌ Info: Recompiling stale cache file C:\Users\krams\.julia\compiled\v0.7\Autologistic\Gemat.ji for Autologistic [05505b62-b0bd-11e8-1cec-1f3e16cbc293]
└ @ Base loading.jl:1185


In [2]:
Pkg.status()

Project Autologistic v0.1.0
    Status `Project.toml`
  [093fc24a] LightGraphs v1.1.0
  [295af30f] Revise v0.7.11
  [10745b16] Statistics 


In [4]:
#-- make an ALmodel object using the full construtor --

# make response, unary part
y = Array{Bool,2}(round.(rand(10,1)))
u1 = FullUnary(rand(10))
u2 = LinPredUnary(rand(10,3), [1, 2, 4])

# make pairwise part
using LightGraphs
G = Graph(10,20)
p = SimplePairwise(G)

# Call the constructor ALmodel(Y,unary,pairwise,centering,coding,labels)
M1 = ALmodel(y, u1, p, none, (-1,1), ("low","high"))
M2 = ALmodel(y, u2, p, none, (-1,1), ("low","high"))
display(M1)
display(M2)

ALmodel{FullUnary{Array{Float64,1}},SimplePairwise{Int64},Centering}(Bool[false; false; … ; false; true], Real[0.761189, 0.107374, 0.698832, 0.825431, 0.981673, 0.870191, 0.224782, 0.368622, 0.402043, 0.763283], Real[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], none::Centering = 0, (-1, 1), ("low", "high"))

ALmodel{LinPredUnary{Array{Float64,2},Array{Int64,1}},SimplePairwise{Int64},Centering}(Bool[false; false; … ; false; true], Real[1.33624, 2.69975, 2.25546, 4.43709, 2.71259, 0.932166, 4.82817, 4.06181, 3.71413, 3.72131], Real[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], none::Centering = 0, (-1, 1), ("low", "high"))

In [6]:
M2.unary.β

3-element Array{Int64,1}:
 1
 2
 4

In [24]:
Matrix(adjacency_matrix(M.pairwise.G))

10×10 Array{Int64,2}:
 0  0  0  0  1  1  0  1  0  0
 0  0  1  0  0  0  0  1  1  0
 0  1  0  1  0  1  0  1  1  0
 0  0  1  0  0  0  0  1  1  1
 1  0  0  0  0  0  0  0  1  0
 1  0  1  0  0  0  1  1  1  1
 0  0  0  0  0  1  0  0  0  1
 1  1  1  1  0  1  0  0  0  0
 0  1  1  1  1  1  0  0  0  1
 0  0  0  1  0  1  1  0  1  0

In [6]:
show(M.unary)

Real[0.4698, 0.803123, 0.209755, 0.380925, 0.437959, 0.619664, 0.476342, 0.325252, 0.389736, 0.0396918]